In [2]:
%matplotlib inline
from river.datasets import synth
from river import drift
from clustream import CluStream
from sklearn.metrics import silhouette_score
import math
import numpy as np
import matplotlib.pyplot as plt
from river.utils import dict2numpy
from scipy.spatial import distance_matrix
from time import time
from river import stream
import pandas as pd


In [3]:
ds = pd.read_csv('./datasets/RBF speed 20L/rbf_speed_20L.csv')
ds.head()

,X1,X2,.class
0,0.840033,0.579297,3
1,0.513139,0.534139,2
2,0.845255,0.654903,3
3,0.280315,0.542376,1
4,0.528570,0.576430,2


In [ ]:
learner = CluStream(n_macro_clusters=3,max_micro_clusters=20,time_window=250,micro_cluster_r_factor=0.09)
#test = drift.PageHinkley(30,delta=0.01,threshold=0.05,alpha= 0.9999,mode='both')



In [ ]:
loop = 0
last_dist = 0
dist = 0

for x,y in stream.iter_pandas(ds,y):
    if loop < 100 :
        learner = learner.learn_one(x)
        
    else :

        learner = learner.learn_one(x)

        print(dist)
        test.update(dist )
        
        if test.drift_detected : 
            #print(f"Drift Detected at instance: {loop}")
            pass
        
        
            
        
    loop += 1
    last_dist = dist

